In [6]:
%matplotlib inline

import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.stats import pearsonr
from scipy.stats import spearmanr

from scipy.cluster.hierarchy import linkage
import scipy.spatial.distance as ssd
from matplotlib.pyplot import rc_context

import seaborn as sns
import sys
from scipy.sparse import hstack, csr_matrix
from sklearn.feature_selection import mutual_info_classif
import os
from cnmf import cNMF, load_df_from_npz
import harmonypy
sys.path.append('../../../Code/')
# from utils import TCAT

from sklearn import metrics
from utils import read_dataset_log
import harmonypy as hm
from sklearn.preprocessing import MinMaxScaler
from adjustText import adjust_text
import random


In [7]:
from datashader.mpl_ext import dsshow
import datashader as ds

import matplotlib.colors
cmap_density = plt.cm.hsv
cmaplist = [cmap_density(i) for i in range(cmap_density.N)]
cmaplist = cmaplist[0:round(len(cmaplist)*0.7)]
cmaplist.reverse()
cmap_density = matplotlib.colors.LinearSegmentedColormap.from_list('', cmaplist, cmap_density.N)

In [3]:
from sklearn.model_selection import KFold


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

from scipy.stats import fisher_exact
from scipy.stats import ttest_ind


import pickle

In [8]:
figdir = '../../../Figures/Components'

In [9]:
params = read_dataset_log('Dataset Paths')
params.index = params['dataset']
params

,dataset,usage_fn,gene_scores_fn,gene_tpm_fn,tcat_fn,tcat_fn_withsingletons,manual_gating_fn,manual_activation_gating_fn,activation_score_fn,metadata_fn,...,k,dt,processed_forcnmf_fn,tpm_counts_for_cnmf_fn,raw_counts_filt_fn,raw_counts_fn,dataset_type,tissue_type,context_label,tcat_fn_old
dataset,,,,,,,,,,,,,,,,,,,,,
AMP-RA,AMP-RA,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMPRA....,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMPRA....,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMPRA....,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,NaN,/data/srlab1/TCAT/Data/PerDataset/AMPRA/Manual...,/data/srlab1/TCAT/Data/PerDataset/AMPRA/Activa...,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMP_AD...,...,34.0,0_15,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMP_AD...,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMP_AD...,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMP_AD...,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMP_AD...,discovery,Synovium,RA+OA,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...
Pan-Cancer,Pan-Cancer,/data/srlab1/TCAT/Data/PerDataset/Pancancer/Pa...,/data/srlab1/TCAT/Data/PerDataset/Pancancer/Pa...,/data/srlab1/TCAT/Data/PerDataset/Pancancer/Pa...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,NaN,NaN,/data/srlab1/TCAT/Data/PerDataset/Pancancer/Ac...,/data/srlab1/TCAT/Data/PerDataset/Pancancer/pa...,...,38.0,0.15,/data/srlab1/TCAT/Data/PerDataset/Pancancer/pa...,NaN,/data/srlab1/TCAT/Data/PerDataset/Pancancer/pa...,/data/srlab1/TCAT/Data/PerDataset/Pancancer/pa...,discovery,Pan-Tissue,Cancer+Healthy,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...
TBRU,TBRU,/data/srlab1/TCAT/Data/PerDataset/TBRU/TBRU.20...,/data/srlab1/TCAT/Data/PerDataset/TBRU/TBRU.20...,/data/srlab1/TCAT/Data/PerDataset/TBRU/TBRU.20...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,NaN,/data/srlab1/TCAT/Data/PerDataset/TBRU/ManualG...,/data/srlab1/TCAT/Data/PerDataset/TBRU/Activat...,/data/srlab1/TCAT/Data/PerDataset/TBRU/tbru_ex...,...,36.0,0_20,/data/srlab1/TCAT/Data/PerDataset/TBRU/tbru_ex...,/data/srlab1/TCAT/Data/PerDataset/TBRU/tbru_ex...,/data/srlab1/TCAT/Data/PerDataset/TBRU/tbru_ex...,/data/srlab1/TCAT/Data/PerDataset/TBRU/tbru_ex...,discovery,Blood,Healthy,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...
HIV-Vaccine,HIV-Vaccine,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/HIVV...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/HIVV...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/HIVV...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/Manu...,NaN,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/Acti...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoe...,...,31.0,0_15,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoe...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoe...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoe...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoe...,discovery,Blood,Post-Vaccine+Healthy,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...
UK-Covid,UK-Covid,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/UKCO...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/UKCO...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/UKCO...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/Manu...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/Manu...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/Acti...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/hani...,...,44.0,0_20,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/hani...,NaN,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/hani...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/hani...,discovery,Blood,Covid-19+Healthy,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...
COMBAT,COMBAT,/data/srlab1/TCAT/Data/PerDataset/COMBAT/COMBA...,/data/srlab1/TCAT/Data/PerDataset/COMBAT/COMBA...,/data/srlab1/TCAT/Data/PerDataset/COMBAT/COMBA...,/data/srlab1/TCAT/Data/TCA

In [10]:
gep_info = read_dataset_log('cGEP_Name')
rename_map = dict(zip(gep_info['cGep_Name'], gep_info['Short_Name']))


In [12]:
# AIM-associated GEPs, excluding GEPs used in other contexts
activation_geps = sorted([
 'CTLA4/CD38',
 'NME1/FABP5',
 # 'Exhaustion',
 'ICOS/CD38',
 'OX40/EBI3',
 'TIMD4/TIM3',
 'Multi-Cytokine',
 'BCL2/FAM13A',
  'RGCC/MYADM'
])
print(len(activation_geps))
activation_geps

8


['BCL2/FAM13A',
 'CTLA4/CD38',
 'ICOS/CD38',
 'Multi-Cytokine',
 'NME1/FABP5',
 'OX40/EBI3',
 'RGCC/MYADM',
 'TIMD4/TIM3']

In [13]:
usage_all = {}
usage_norm_all = {}
meta_all = {}
activation_all = {}

In [14]:
ds_list = params.index

In [15]:
for dataset in ds_list:
    print(dataset)
    usage = pd.read_csv(params.loc[dataset, 'tcat_fn'], index_col = 0, sep = '\t')
    usage.columns = usage.columns.map(rename_map)
    usage_norm = usage.div(usage.sum(axis=1), axis=0)
    
    if dataset in ['TBRU', 'HIV-Vaccine', 'Sparks']:
        meta = pd.read_csv(params.loc[dataset, 'metadata_fn'],
                  index_col = 0, sep = '\t')
        
    elif dataset in ['Stimseq']:
        counts_fn = params.loc[dataset, 'raw_counts_filt_fn']
        adata = sc.read(counts_fn)
        meta = adata.obs.loc[usage_norm.index, :]
        
        # Rescale usage matrix
        scaler = 1e2
        usage = usage*scaler
        
    else:
        meta = pd.read_csv(params.loc[dataset, 'metadata_fn'].replace('metadata', 'metadata_PhenotypeAssociations'),
                      index_col = 0, sep = '\t')
    meta = meta.loc[usage_norm.index, ]
    

    usage_all[dataset] = usage
    usage_norm_all[dataset] = usage_norm
    meta_all[dataset] = meta


AMP-RA
Pan-Cancer


/PHShome/mc1070/anaconda3/envs/cnmf_env2v2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


TBRU
HIV-Vaccine
UK-Covid


/PHShome/mc1070/anaconda3/envs/cnmf_env2v2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (18,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


COMBAT
Pan-Tissue


/PHShome/mc1070/anaconda3/envs/cnmf_env2v2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Sparks


/PHShome/mc1070/anaconda3/envs/cnmf_env2v2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Stimseq
Only considering the two last: ['.doubletfilt', '.h5ad'].
Only considering the two last: ['.doubletfilt', '.h5ad'].


In [16]:
gating_act_all = {}

for dataset in ['COMBAT', 'Sparks', 'Stimseq']:
    gating_act_all[dataset] = pd.read_csv(params.loc[dataset, 'manual_activation_gating_fn'],
                             index_col = 0, sep = '\t')

In [17]:
gating_all = {}

for dataset in ['COMBAT', 'Sparks', 'Stimseq']:
    gating_all[dataset] = pd.read_csv(params.loc[dataset, 'manual_gating_fn'],
                             index_col = 0, sep = '\t')

gating_class_all = {}
for dataset in ['COMBAT', 'Sparks', 'Stimseq']:
    gating = gating_all[dataset]['0'].copy()
    gating_class = pd.Series(np.nan, index = gating.index)
    
    gating_class.loc[gating.str.contains('CD4')] = 'CD4 Conv'
    gating_class.loc[gating=='Treg'] = 'CD4 Treg'
    gating_class.loc[gating.str.contains('CD8')] = 'CD8 Conv'
    gating_class.loc[gating_class.isna()] = 'Other'

    gating_class_all[dataset] = gating_class

In [18]:
activation_geps

['BCL2/FAM13A',
 'CTLA4/CD38',
 'ICOS/CD38',
 'Multi-Cytokine',
 'NME1/FABP5',
 'OX40/EBI3',
 'RGCC/MYADM',
 'TIMD4/TIM3']

In [20]:
'''
Run one step of forward selection for a single dataset. 
Predict gating from activation score using a given gep_list + each other possible gep.
'''
def run_step(dataset, step, gep_list, k = 9, test_size = 0.1, lineage = None,
             random_states = [1],
             usage_norm_all = usage_norm_all, gating_act_all = gating_act_all, 
             gating_class_all = gating_class_all, activation_geps = activation_geps):
    # Load dataset specific results
    usage_norm = usage_norm_all[dataset]
    gating_act = gating_act_all[dataset]
    usage_for_model = usage_norm[activation_geps]
    overlap = sorted(list(set(usage_for_model.index).intersection(gating_act.index)))
    
    # Subset to specific lineages
    if lineage != None:
        gating_class = gating_class_all[dataset]
        ind_lineage = gating_class[gating_class==lineage].index
        overlap = sorted(list(set(ind_lineage).intersection(overlap)))
        
        gating_act = gating_act.loc[overlap, :]
        usage_for_model = usage_for_model.loc[overlap, :]
        
    if dataset == 'Stimseq':
        meta = meta_all[dataset]
        overlap = meta[meta['Sort_Status'].isin(['+', '-'])].index
        meta.loc[overlap, :]['Sort_Status']=='+'
        label_all = meta.loc[overlap, :]['Sort_Status']=='+'
    else:
        label_all = gating_act.loc[overlap, 'Activated']=='Activated'

    # # Hold out testing set if test_size != None
    # if test_size:
    #     X_train, X_test, y_train, y_test = train_test_split(usage_for_model.loc[overlap, :], 
    #                                                         label_all[overlap], test_size = 0.1, random_state=1)
    # else:
    
    X_train = usage_for_model.loc[overlap, :]
    y_train = label_all[overlap]

    auc_step_df = pd.DataFrame() # Df for this step for this dataset

    # Loop through all GEPs
    for gep in sorted(set(usage_for_model.columns).difference(gep_list)):
        # Calculate activation score (summed usages)
        score_train = pd.Series(np.sum(X_train[gep_list + [gep]], axis = 1), index = X_train.index)

        auc_train_folds, auc_test_folds = [], []
        
        auc_train = roc_auc_score(y_train, score_train)
        auc_test = auc_train

        auc_train_folds += [auc_train]
        auc_test_folds += [auc_test]


        auc_step_df = auc_step_df.append(pd.DataFrame([','.join(gep_list + [gep]), gep, step, 
                          np.mean(auc_train_folds), np.mean(auc_test_folds)], 
                          index = ['geps', 'new_gep', 'step',
                                'auc_%s_train_cv_mean' % dataset , 'auc_%s_test_cv_mean' % dataset]).T)
    return auc_step_df

In [28]:
auc_stepwise

,geps,new_gep,step,auc_Sparks_train_cv_mean,auc_Sparks_test_cv_mean,auc_COMBAT_train_cv_mean,auc_COMBAT_test_cv_mean,auc_test_cv_Mean,auc_train_cv_Mean
0,ICOS/CD38,ICOS/CD38,0,0.776303,0.776303,0.890078,0.890078,0.833191,0.833191
1,"ICOS/CD38,CTLA4/CD38",CTLA4/CD38,1,0.821747,0.821747,0.900205,0.900205,0.860976,0.860976
2,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3",TIMD4/TIM3,2,0.811385,0.811385,0.926657,0.926657,0.869021,0.869021
3,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3,OX40/EBI3",OX40/EBI3,3,0.81751,0.81751,0.919685,0.919685,0.868598,0.868598
4,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3,OX40/EBI3,Mult...",Multi-Cytokine,4,0.808385,0.808385,0.908399,0.908399,0.858392,0.858392
5,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3,OX40/EBI3,Mult...",RGCC/MYADM,5,0.802554,0.802554,0.895381,0.895381,0.848968,0.848968
6,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3,OX40/EBI3,Mult...",NME1/FABP5,6,0.747516,0.747516,0.892443,0.892443,0.819980,0.819980
7,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3,OX40/EBI3,Mult...",BCL2/FAM13A,7,0.657149,0.657149,0.797966,0.797966,0.727558,0.727558


# Feature selection

No heldout data, k = 10, all lineages

In [21]:
activation_geps

['BCL2/FAM13A',
 'CTLA4/CD38',
 'ICOS/CD38',
 'Multi-Cytokine',
 'NME1/FABP5',
 'OX40/EBI3',
 'RGCC/MYADM',
 'TIMD4/TIM3']

In [22]:
test_datasets = ['Sparks', 'COMBAT']

In [23]:
auc_all = pd.DataFrame()
auc_stepwise = pd.DataFrame()
auc_last = 0
loop = True
step = 0
gep_list = []

while (loop):
    print(step)
    auc_step = {}

    for dataset in test_datasets:
        print(dataset)
        auc_step[dataset] = run_step(dataset, step, gep_list)

    auc_step_all = pd.merge(auc_step[test_datasets[0]], auc_step[test_datasets[1]], on = ['geps', 'new_gep', 'step'])
    auc_step_all['auc_test_cv_Mean'] = np.mean(auc_step_all[['auc_%s_test_cv_mean'% test_datasets[0], 
                                                 'auc_%s_test_cv_mean'% test_datasets[1]]], axis = 1)
    auc_step_all['auc_train_cv_Mean'] = np.mean(auc_step_all[['auc_%s_train_cv_mean'% test_datasets[0], 
                                                 'auc_%s_train_cv_mean'% test_datasets[1]]], axis = 1)
    auc_all = auc_all.append(auc_step_all)
    
    # Add GEP with highest mean AUC across datasets
    top_gep = auc_step_all.sort_values('auc_test_cv_Mean', ascending = False).reset_index(drop = True).head(1)
    add_gep = top_gep.loc[0, 'new_gep']
    auc_gep_test = top_gep.loc[0, 'auc_test_cv_Mean']
    auc_gep_train = top_gep.loc[0, 'auc_train_cv_Mean']
    
    # Loop through all activation GEPs regardless of AUC improvement
    loop = len(set(activation_geps).difference(gep_list))>1 #auc_gep_test > auc_last
    gep_list += [add_gep]
    auc_stepwise = auc_stepwise.append(top_gep)
    auc_last = auc_gep_test
    step +=1
    
    print(add_gep, auc_gep_test, auc_gep_train)
auc_stepwise = auc_stepwise.reset_index(drop = True)

0
Sparks
COMBAT
ICOS/CD38 0.8331907059363326 0.8331907059363326
1
Sparks
COMBAT
CTLA4/CD38 0.8609760366578083 0.8609760366578083
2
Sparks
COMBAT
TIMD4/TIM3 0.8690208774586765 0.8690208774586765
3
Sparks
COMBAT
OX40/EBI3 0.8685977155208637 0.8685977155208637
4
Sparks
COMBAT
Multi-Cytokine 0.8583920429316892 0.8583920429316892
5
Sparks
COMBAT
RGCC/MYADM 0.8489677212801423 0.8489677212801423
6
Sparks
COMBAT
NME1/FABP5 0.8199795079643055 0.8199795079643055
7
Sparks
COMBAT
BCL2/FAM13A 0.7275575235418635 0.7275575235418635


In [32]:
auc_all = pd.DataFrame()
auc_stepwise = pd.DataFrame()
auc_last = 0
loop = True
step = 0
gep_list = []

while (loop):
    print(step)
    auc_step = {}

    for dataset in test_datasets:
        print(dataset)
        auc_step[dataset] = run_step(dataset, step, gep_list, k  = 10, test_size = None)

    auc_step_all = pd.merge(auc_step[test_datasets[0]], auc_step[test_datasets[1]], on = ['geps', 'new_gep', 'step'])
    auc_step_all['auc_test_cv_Mean'] = np.mean(auc_step_all[['auc_%s_test_cv_mean'% test_datasets[0], 
                                                 'auc_%s_test_cv_mean'% test_datasets[1]]], axis = 1)
    auc_step_all['auc_train_cv_Mean'] = np.mean(auc_step_all[['auc_%s_train_cv_mean'% test_datasets[0], 
                                                 'auc_%s_train_cv_mean'% test_datasets[1]]], axis = 1)
    auc_all = auc_all.append(auc_step_all)
    
    # Add GEP with highest mean AUC across datasets
    top_gep = auc_step_all.sort_values('auc_test_cv_Mean', ascending = False).reset_index(drop = True).head(1)
    add_gep = top_gep.loc[0, 'new_gep']
    auc_gep_test = top_gep.loc[0, 'auc_test_cv_Mean']
    auc_gep_train = top_gep.loc[0, 'auc_train_cv_Mean']
    
    # Loop through all activation GEPs regardless of AUC improvement
    loop = len(set(activation_geps).difference(gep_list))>1 #auc_gep_test > auc_last
    gep_list += [add_gep]
    auc_stepwise = auc_stepwise.append(top_gep)
    auc_last = auc_gep_test
    step +=1
    
    print(add_gep, auc_gep_test, auc_gep_train)
auc_stepwise = auc_stepwise.reset_index(drop = True)

0
Sparks
COMBAT
ICOS/CD38 0.8328741389366279 0.8331865037103563
1
Sparks
COMBAT
CTLA4/CD38 0.8607018958747956 0.860972303508095
2
Sparks
COMBAT
TIMD4/TIM3 0.8687331029794585 0.8690169875761702
3
Sparks
COMBAT
OX40/EBI3 0.8682796231680525 0.8685934028223071
4
Sparks
COMBAT
Multi-Cytokine 0.8580989940793182 0.8583880686210243
5
Sparks
COMBAT
RGCC/MYADM 0.8486256691779178 0.8489630916367634
6
Sparks
COMBAT
NME1/FABP5 0.8196792678448026 0.8199754014137464
7
Sparks
COMBAT
BCL2/FAM13A 0.7273083787676593 0.7275540505732763


In [24]:
auc_stepwise[['new_gep', 'auc_test_cv_Mean', 'auc_COMBAT_test_cv_mean', 'auc_Sparks_test_cv_mean']]

,new_gep,auc_test_cv_Mean,auc_COMBAT_test_cv_mean,auc_Sparks_test_cv_mean
0,ICOS/CD38,0.833191,0.890078,0.776303
1,CTLA4/CD38,0.860976,0.900205,0.821747
2,TIMD4/TIM3,0.869021,0.926657,0.811385
3,OX40/EBI3,0.868598,0.919685,0.81751
4,Multi-Cytokine,0.858392,0.908399,0.808385
5,RGCC/MYADM,0.848968,0.895381,0.802554
6,NME1/FABP5,0.819980,0.892443,0.747516
7,BCL2/FAM13A,0.727558,0.797966,0.657149


In [25]:
auc_stepwise.to_csv('../../../Data/TCAT/AUC_ActivationPrediction_AIMAssocGEP_StepwiseCOMBATSparks_20240424.tsv', 
                   sep = '\t')

In [26]:
auc_stepwise.head()

,geps,new_gep,step,auc_Sparks_train_cv_mean,auc_Sparks_test_cv_mean,auc_COMBAT_train_cv_mean,auc_COMBAT_test_cv_mean,auc_test_cv_Mean,auc_train_cv_Mean
0,ICOS/CD38,ICOS/CD38,0,0.776303,0.776303,0.890078,0.890078,0.833191,0.833191
1,"ICOS/CD38,CTLA4/CD38",CTLA4/CD38,1,0.821747,0.821747,0.900205,0.900205,0.860976,0.860976
2,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3",TIMD4/TIM3,2,0.811385,0.811385,0.926657,0.926657,0.869021,0.869021
3,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3,OX40/EBI3",OX40/EBI3,3,0.81751,0.81751,0.919685,0.919685,0.868598,0.868598
4,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3,OX40/EBI3,Mult...",Multi-Cytokine,4,0.808385,0.808385,0.908399,0.908399,0.858392,0.858392


# Apply to all datasets

## Top 3 GEPs

In [37]:
auc_stepwise

,geps,new_gep,step,auc_Sparks_train_cv_mean,auc_Sparks_test_cv_mean,auc_COMBAT_train_cv_mean,auc_COMBAT_test_cv_mean,auc_test_cv_Mean,auc_train_cv_Mean
0,ICOS/CD38,ICOS/CD38,0,0.776295,0.775668,0.890078,0.89008,0.832874,0.833187
1,"ICOS/CD38,CTLA4/CD38",CTLA4/CD38,1,0.821739,0.8212,0.900205,0.900204,0.860702,0.860972
2,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3",TIMD4/TIM3,2,0.811377,0.810815,0.926657,0.926652,0.868733,0.869017
3,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3,OX40/EBI3",OX40/EBI3,3,0.817501,0.816873,0.919685,0.919686,0.868280,0.868593
4,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3,OX40/EBI3,Mult...",Multi-Cytokine,4,0.808377,0.807798,0.908399,0.9084,0.858099,0.858388
5,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3,OX40/EBI3,Mult...",RGCC/MYADM,5,0.802545,0.801869,0.895381,0.895382,0.848626,0.848963
6,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3,OX40/EBI3,Mult...",NME1/FABP5,6,0.747508,0.746909,0.892443,0.89245,0.819679,0.819975
7,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3,OX40/EBI3,Mult...",BCL2/FAM13A,7,0.657142,0.656619,0.797966,0.797998,0.727308,0.727554


In [41]:
pd.Index(auc_stepwise['new_gep'])

Index(['ICOS/CD38', 'CTLA4/CD38', 'TIMD4/TIM3', 'OX40/EBI3', 'Multi-Cytokine',
       'RGCC/MYADM', 'NME1/FABP5', 'BCL2/FAM13A'],
      dtype='object', name='new_gep')

In [42]:
gep_list = ['ICOS/CD38', 'CTLA4/CD38', 'TIMD4/TIM3']
gep_list = sorted(gep_list)
gep_list

['CTLA4/CD38', 'ICOS/CD38', 'TIMD4/TIM3']

In [43]:
name = 'ActivationScore.%s.Sum_CTLA4-ICOS-TIMD4.20240223.tsv'

In [44]:
ds_list = params.index

In [431]:
for dataset in ds_list:
    print(dataset)
    
    out_dir = os.path.dirname(params.loc[dataset, 'raw_counts_fn'])
    out_fn = '%s/%s' % (out_dir, name % dataset)
    print(out_fn)
    
    usage_norm = usage_norm_all[dataset]
    usage_for_model = usage_norm[gep_list]

    score_all = np.sum(usage_for_model.loc[:, gep_list], axis = 1)  
    score_all = pd.Series(score_all, index = usage_for_model.index)

    activation = pd.DataFrame(score_all, columns = ['Activation_Score'])
    activation.to_csv(out_fn, sep = '\t')
    
    print(activation.shape)

AMP-RA
/data/srlab1/TCAT/Data/PerDataset/AMPRA/ActivationScore.AMP-RA.Sum_CTLA4-ICOS-TIMD4.20240223.tsv
(94048, 1)
Pan-Cancer
/data/srlab1/TCAT/Data/PerDataset/Pancancer/ActivationScore.Pan-Cancer.Sum_CTLA4-ICOS-TIMD4.20240223.tsv
(215035, 1)
TBRU
/data/srlab1/TCAT/Data/PerDataset/TBRU/ActivationScore.TBRU.Sum_CTLA4-ICOS-TIMD4.20240223.tsv
(500089, 1)
HIV-Vaccine
/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/ActivationScore.HIV-Vaccine.Sum_CTLA4-ICOS-TIMD4.20240223.tsv
(73259, 1)
UK-Covid
/data/srlab1/TCAT/Data/PerDataset/UKCOVID/ActivationScore.UK-Covid.Sum_CTLA4-ICOS-TIMD4.20240223.tsv
(273516, 1)
COMBAT
/data/srlab1/TCAT/Data/PerDataset/COMBAT/ActivationScore.COMBAT.Sum_CTLA4-ICOS-TIMD4.20240223.tsv
(399599, 1)
Pan-Tissue
/data/srlab1/TCAT/Data/PerDataset/XTissueImmuneAtlas/ActivationScore.Pan-Tissue.Sum_CTLA4-ICOS-TIMD4.20240223.tsv
(154314, 1)
Sparks
/data/srlab1/TCAT/Data/PerDataset/Sparks2023/ActivationScore.Sparks.Sum_CTLA4-ICOS-TIMD4.20240223.tsv
(336739, 1)
Stimseq
/data/srlab1/T

## Top 4 GEPs

In [46]:
auc_stepwise

,geps,new_gep,step,auc_Sparks_train_cv_mean,auc_Sparks_test_cv_mean,auc_COMBAT_train_cv_mean,auc_COMBAT_test_cv_mean,auc_test_cv_Mean,auc_train_cv_Mean
0,ICOS/CD38,ICOS/CD38,0,0.776295,0.775668,0.890078,0.89008,0.832874,0.833187
1,"ICOS/CD38,CTLA4/CD38",CTLA4/CD38,1,0.821739,0.8212,0.900205,0.900204,0.860702,0.860972
2,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3",TIMD4/TIM3,2,0.811377,0.810815,0.926657,0.926652,0.868733,0.869017
3,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3,OX40/EBI3",OX40/EBI3,3,0.817501,0.816873,0.919685,0.919686,0.868280,0.868593
4,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3,OX40/EBI3,Mult...",Multi-Cytokine,4,0.808377,0.807798,0.908399,0.9084,0.858099,0.858388
5,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3,OX40/EBI3,Mult...",RGCC/MYADM,5,0.802545,0.801869,0.895381,0.895382,0.848626,0.848963
6,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3,OX40/EBI3,Mult...",NME1/FABP5,6,0.747508,0.746909,0.892443,0.89245,0.819679,0.819975
7,"ICOS/CD38,CTLA4/CD38,TIMD4/TIM3,OX40/EBI3,Mult...",BCL2/FAM13A,7,0.657142,0.656619,0.797966,0.797998,0.727308,0.727554


In [47]:
gep_list = ['ICOS/CD38', 'CTLA4/CD38', 'TIMD4/TIM3', 'OX40/EBI3']
gep_list = sorted(gep_list)
gep_list

['CTLA4/CD38', 'ICOS/CD38', 'OX40/EBI3', 'TIMD4/TIM3']

In [646]:
name = 'ActivationScore.%s.Sum_CTLA4-ICOS-OX40-TIMD4.20240225.tsv'

In [647]:
ds_list = params.index

In [648]:
for dataset in ds_list:
    print(dataset)
    
    out_dir = os.path.dirname(params.loc[dataset, 'raw_counts_fn'])
    out_fn = '%s/%s' % (out_dir, name % dataset)
    print(out_fn)
    
    usage_norm = usage_norm_all[dataset]
    usage_for_model = usage_norm[gep_list]

    score_all = np.sum(usage_for_model.loc[:, gep_list], axis = 1)  
    score_all = pd.Series(score_all, index = usage_for_model.index)

    activation = pd.DataFrame(score_all, columns = ['Activation_Score'])
    activation.to_csv(out_fn, sep = '\t')
    
    print(activation.shape)

AMP-RA
/data/srlab1/TCAT/Data/PerDataset/AMPRA/ActivationScore.AMP-RA.Sum_CTLA4-ICOS-OX40-TIMD4.20240225.tsv
(94048, 1)
Pan-Cancer
/data/srlab1/TCAT/Data/PerDataset/Pancancer/ActivationScore.Pan-Cancer.Sum_CTLA4-ICOS-OX40-TIMD4.20240225.tsv
(215035, 1)
TBRU
/data/srlab1/TCAT/Data/PerDataset/TBRU/ActivationScore.TBRU.Sum_CTLA4-ICOS-OX40-TIMD4.20240225.tsv
(500089, 1)
HIV-Vaccine
/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/ActivationScore.HIV-Vaccine.Sum_CTLA4-ICOS-OX40-TIMD4.20240225.tsv
(73259, 1)
UK-Covid
/data/srlab1/TCAT/Data/PerDataset/UKCOVID/ActivationScore.UK-Covid.Sum_CTLA4-ICOS-OX40-TIMD4.20240225.tsv
(273516, 1)
COMBAT
/data/srlab1/TCAT/Data/PerDataset/COMBAT/ActivationScore.COMBAT.Sum_CTLA4-ICOS-OX40-TIMD4.20240225.tsv
(399599, 1)
Pan-Tissue
/data/srlab1/TCAT/Data/PerDataset/XTissueImmuneAtlas/ActivationScore.Pan-Tissue.Sum_CTLA4-ICOS-OX40-TIMD4.20240225.tsv
(154314, 1)
Sparks
/data/srlab1/TCAT/Data/PerDataset/Sparks2023/ActivationScore.Sparks.Sum_CTLA4-ICOS-OX40-TIMD4.2024022